<a href="https://colab.research.google.com/github/aajlani2023/main/blob/main/GamePulse_CNN_LSTM_Notebook_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🏈 GamePulse™ CNN + LSTM Training Notebook
Ce notebook permet de s'exercer à l'entraînement d'un modèle CNN + LSTM
pour **analyser la performance et la fatigue des joueurs NFL** à partir de données physiologiques simulées.

**Pipeline :**
1. Simulation de données capteurs (ECG, HRV, IMU)
2. Prétraitement (fenêtrage + spectrogrammes pour CNN)
3. CNN pour extraire des features
4. LSTM pour analyser la séquence des features
5. Prédiction : `fatigue_score` et recommandation `Play/Rest/Substitute`


In [1]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import random


In [2]:

# Paramètres de simulation
num_players = 5
signal_length = 3000  # points simulés par joueur (~5 min à 10 Hz)
fs = 10  # fréquence d'échantillonnage Hz

def simulate_player_signals(num_samples=signal_length, fs=10):
    t = np.arange(num_samples) / fs
    # ECG-like signal (simple sine with noise)
    ecg = np.sin(2*np.pi*1.2*t) + 0.1*np.random.randn(num_samples)
    # HRV: inverse of synthetic RR variability
    hrv = 60 + 5*np.sin(2*np.pi*0.1*t) + np.random.randn(num_samples)
    # IMU (acceleration magnitude)
    imu = np.abs(np.sin(2*np.pi*0.3*t + np.random.rand()) + 0.2*np.random.randn(num_samples))
    return np.vstack([ecg, hrv, imu]).T

# Génération des données pour plusieurs joueurs
signals = []
labels = []
for player in range(num_players):
    sig = simulate_player_signals()
    signals.append(sig)
    # Fatigue simulée (0=low,1=rest,2=substitute)
    fatigue_label = random.choice([0,1,2])
    labels.append(fatigue_label)

signals = np.array(signals)
labels = np.array(labels)
signals.shape, labels


((5, 3000, 3), array([1, 2, 1, 0, 0]))

In [3]:

from scipy.signal import spectrogram

def compute_spectrogram(signal_1d, fs=10):
    f, t, Sxx = spectrogram(signal_1d, fs=fs, nperseg=64, noverlap=32)
    return Sxx

# Conversion des signaux ECG en spectrogrammes pour CNN
spectrograms = []
for sig in signals:
    ecg_sig = sig[:,0]
    Sxx = compute_spectrogram(ecg_sig, fs)
    spectrograms.append(Sxx)

spectrograms = np.array(spectrograms)
spectrograms = spectrograms[..., np.newaxis]  # ajout d'un canal pour CNN
spectrograms.shape


(5, 33, 92, 1)

In [4]:

X_train, X_test, y_train, y_test = train_test_split(spectrograms, labels, test_size=0.3, random_state=42)
print(X_train.shape, y_train.shape)


(3, 33, 92, 1) (3,)


In [5]:

model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.RepeatVector(10),  # simulation d'une séquence temporelle
    layers.LSTM(32),
    layers.Dense(3, activation='softmax')  # 3 classes: Play, Rest, Substitute
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 31, 90, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 45, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 43, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 21, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4032)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 10, 4032)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │       520,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 525,219 (2.00 MB)

 Trainable params: 525,219 (2.00 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:

history = model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 379ms/step - accuracy: 0.0000e+00 - loss: 1.1912 - val_accuracy: 0.0000e+00 - val_loss: 1.9490
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6111 - loss: 0.7129 - val_accuracy: 0.0000e+00 - val_loss: 2.5435
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.7778 - loss: 0.6557 - val_accuracy: 0.0000e+00 - val_loss: 2.8389
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6111 - loss: 0.8714 - val_accuracy: 0.0000e+00 - val_loss: 2.6730
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.7778 - loss: 0.5433 - val_accuracy: 0.0000e+00 - val_loss: 2.5867
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.7778 - loss: 0.5338 - val_accuracy: 0.0000e+00 - val_loss: 2.4709
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.6111 - loss: 0.7140 - val_accuracy: 0.0000e+00 - val_loss: 2.3759
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6111 - loss: 0.6978 - val_acc

In [7]:

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.0000e+00 - loss: 2.4587
Test Accuracy: 0.00%


In [8]:

pred = model.predict(X_test[:1])
classes = ['Play','Rest','Substitute']
print("Prediction:", classes[np.argmax(pred)])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Prediction: Rest
